In [2]:
from langchain_openai import OpenAIEmbeddings
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import pickle
import zlib

with open("./openai.key", "r") as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()

In [4]:
# load compressed model
with open("./classification_models/best.pkl.zlib", "rb") as f:
    compressed = f.read()
    automl = pickle.loads(zlib.decompress(compressed))
    print(automl)
# load model list
ROUNDS = 10
models = list(pd.read_feather("./datasets/models.feather")["id"])
dataset = "revised_dataset"
# embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

ModuleNotFoundError: No module named 'autosklearn'

In [3]:
for model in tqdm(models):
    model_name = model.replace("/", "_")
    for i in range(ROUNDS):
        if os.path.exists(f"./results/tagged/{dataset}_{model_name}_{i}.pkl"):
            print(f"found {dataset}_{model_name}_{i}")
            continue
        if os.path.exists(f"./results/{dataset}_{model_name}_{i}.pkl"):
            df = pd.read_pickle(f"./results/{dataset}_{model_name}_{i}.pkl")
            df["model"] = model
            df["round"] = i
            df['sentence'] = df['Prompt'] + ' ' + df['answer']
            df["embeddings"] = embeddings.embed_documents(df['sentence'])
            df["embeddings"] = df["embeddings"].apply(lambda x: np.array(x, dtype=np.float64))
            df["tag"] = automl.predict(df["embeddings"].values.tolist())
            # save
            df = df.drop(columns=["embeddings", "sentence"])
            df.to_pickle(f"./results/tagged/{dataset}_{model_name}_{i}.pkl")

  0%|          | 0/89 [00:00<?, ?it/s]

found revised_dataset_mistralai_mistral-7b-instruct-v0.3_0
found revised_dataset_mistralai_mistral-7b-instruct-v0.3_1
found revised_dataset_mistralai_mistral-7b-instruct-v0.3_2
found revised_dataset_mistralai_mistral-7b-instruct-v0.3_3
found revised_dataset_mistralai_mistral-7b-instruct-v0.3_4
found revised_dataset_mistralai_mistral-7b-instruct-v0.3_5
found revised_dataset_mistralai_mistral-7b-instruct-v0.3_6
found revised_dataset_mistralai_mistral-7b-instruct-v0.3_7
found revised_dataset_mistralai_mistral-7b-instruct-v0.3_8
found revised_dataset_mistralai_mistral-7b-instruct-v0.3_9
found revised_dataset_openchat_openchat-7b_0
found revised_dataset_openchat_openchat-7b_1
found revised_dataset_openchat_openchat-7b_2
found revised_dataset_openchat_openchat-7b_3
found revised_dataset_openchat_openchat-7b_4
found revised_dataset_openchat_openchat-7b_5
found revised_dataset_openchat_openchat-7b_6
found revised_dataset_openchat_openchat-7b_7
found revised_dataset_openchat_openchat-7b_8
found